<h1 align =center> Capstone Project Week 3

<h2 align = center> Segmenting and Clustering neighborhood in Toronto



In [1]:
 pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests 
from bs4 import BeautifulSoup

import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 
from pandas.io.json import json_normalize 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes

import folium 

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


In [4]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade edx-dl 

Requirement already up-to-date: edx-dl in /home/jupyterlab/conda/envs/python/lib/python3.6/site-packages (0.1.10)
Note: you may need to restart the kernel to use updated packages.


In [6]:

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'  

wiki_url = requests.get(url).text 


soup = BeautifulSoup(wiki_url, 'html.parser')
My = soup.find_all('table')[0]

Torontodf = pd.read_html(str(My))[0] 
Torontodf.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


<h2> Cleaning dataframe

In [7]:

Torontodf.replace("Not assigned", np.nan, inplace = True) 


Torontodf.dropna(subset=['Borough'], axis=0, inplace=True)

Torontodf.reset_index(drop=True, inplace =True)


Torontodf.Neighbourhood = np.where(Torontodf.Neighbourhood.isnull(), Torontodf.Borough, Torontodf.Neighbourhood)

Torontoneigh=Torontodf.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda text: ''.join(text.to_string(index=False))).str.replace('(\\n)', ',').reset_index()

<h2> Creating a data frame of the neighbourhoods of Toronto

In [8]:
Torontoneigh_F=Torontoneigh.rename(columns ={'Postcode': 'PostalCode', 'Borough':'Borough', 'Neighbourhood':'Neighborhood'})
Torontoneigh_F.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, ..."
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Clif..."
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
Torontoneigh_F.shape

(103, 3)

In [10]:

NeighLongLat_df = pd.read_csv('http://cocl.us/Geospatial_data')

NeighLongLat_df = NeighLongLat_df.rename(columns={'Postal Code': 'PostalCode'})


Torontoneigh_LatLong = pd.merge(Torontoneigh_F, NeighLongLat_df, how = 'left', on = 'PostalCode')
Torontoneigh_LatLong.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, ...",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Clif...",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


creating a new dataframe of the Borough that contain the word Toronto

In [11]:
Options= ['East Toronto', 'West Toronto', 'Central Toronto', 'Downtown Toronto']
Torontodata = Torontoneigh_LatLong[Torontoneigh_LatLong['Borough'].isin(Options)].reset_index(drop=True)
Torontodata.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Getting the geographical coordinates


In [12]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent='ca_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {},{}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.653963,-79.387207.


Let us visualize Toronto  and the neighborhoods in it.

In [13]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 11)

#add markers to map
for lat, lng, label in zip(Torontodata['Latitude'], Torontodata['Longitude'], Torontodata['Neighborhood']):
    label = folium.Popup(label, parse_html= True)
    folium.CircleMarker([lat, lng], radius =5, popup = label, color = 'blue',
                        fill = True, fill_color ='#3186cc', fill_opacity=0.7,
                        parse_html=False).add_to(map_toronto)
map_toronto

In [14]:
CLIENT_ID="AF1TURMW1I0OLD5O4UCUZM54WNI0PCGO2IQ5D2Z4J5ZKZHOL"
CLIENT_SECRET="2LLHJ45OCRQC4NR0CA1IHO2VBDJJ2GGQ0G4QLI50J2BRJX3J"
VERSION="20190824"

Exploring neighbourhoods in Toronto

In [15]:
def getNearbyVenues(names, latitudes, longitudes, LIMIT=100, radius =500):
    venues_list =[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL    
        url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
             name,
             lat,
             lng,
             v['venue']['name'],
             v['venue']['location']['lat'],
             v['venue']['location']['lng'],
             v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                            'Neighborhood Latitude',
                            'Neighborhood Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category']
    return(nearby_venues)

In [16]:
toronto_venues = getNearbyVenues(names=Torontodata['Neighborhood'],
                                 latitudes = Torontodata['Latitude'],
                                 longitudes = Torontodata['Longitude'])

 The Beaches
 The Danforth West,         Riverdale
 The Beaches West,     India Bazaar
 Studio District
 Lawrence Park
 Davisville North
 North Toronto West
 Davisville
      Moore Park, Summerhill East
       Deer Park,  Forest Hill SE,       Rathnelly,      South Hill, Summerhill West
 Rosedale
    Cabbagetown, St. James Town
 Church and Wellesley
 Harbourfront,  Regent Park
         Ryerson, Garden District
 St. James Town
 Berczy Park
 Central Bay Street
 Adelaide,     King, Richmond
 Harbourfront East,   Toronto Islands,     Union Station
         Design Exchange, Toronto Dominion Centre
 Commerce Court, Victoria Hotel
 Roselawn
 Forest Hill North,  Forest Hill West
     The Annex, North Midtown,     Yorkville
               Harbord, University of Toronto
         Chinatown,       Grange Park, Kensington Market
          CN Tower,     Bathurst Quay,    Island airport, Harbourfront West,  King and Spadina,     Railway Lands,     South Niagara
 Stn A PO Boxes 25 The Esplanade
 First

In [17]:
print(toronto_venues.shape)
toronto_venues.head()

(1699, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [18]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Harbord, University of Toronto",35,35,35,35,35,35
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"High Park, The Junction South",23,23,23,23,23,23
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
"Design Exchange, Toronto Dominion Centre",100,100,100,100,100,100
"Ryerson, Garden District",100,100,100,100,100,100
"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",15,15,15,15,15,15
"Moore Park, Summerhill East",4,4,4,4,4,4



Let us find out how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 239 uniques categories.


Analyzing Each Neighborhood

In [20]:
torontoencode = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
torontoencode['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [torontoencode.columns[-1]] + list(torontoencode.columns[:-1])
torontoencode = torontoencode[fixed_columns]

torontoencode.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [21]:
torontoencode.shape

(1699, 239)


Now, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
torontogrp = torontoencode.groupby('Neighborhood').mean().reset_index()
torontogrp

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Women's Store
0,"Harbord, University of Toronto",0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057143,0.000000,0.057143,0.00,0.000000,0.000000,0.028571,0.028571,0.000000,0.000000,0.057143,0.000000,0.00,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.00,0.000000,0.114286,0.000000,0.00,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.028571,0.028571,0.00,0.00,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.

Next, let us confirm the size

In [23]:
torontogrp.shape

(38, 239)

Let's put the top venues of each neighbourhood in Toronto into a pandas dataframe

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [25]:

num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = torontogrp['Neighborhood']

for ind in np.arange(torontogrp.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(torontogrp.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbord, University of Toronto",Café,Bakery,Bar,Bookstore,Restaurant,Japanese Restaurant,Chinese Restaurant,Beer Bar,Beer Store,Flower Shop
1,"CN Tower, Bathurst Quay, Isla...",Airport Service,Airport Terminal,Airport Lounge,Plane,Sculpture Garden,Boat or Ferry,Harbor / Marina,Coffee Shop,Airport Gate,Airport Food Court
2,"High Park, The Junction South",Café,Mexican Restaurant,Bookstore,Arts & Crafts Store,Flea Market,Cajun / Creole Restaurant,Park,Speakeasy,Fried Chicken Joint,Bar
3,"Brockton, Exhibition Place, Parkdale ...",Breakfast Spot,Café,Coffee Shop,Furniture / Home Store,Intersection,Caribbean Restaurant,Restaurant,Stadium,Climbing Gym,Bar
4,"Chinatown, Grange Park, Kensing...",Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bakery,Vietnamese Restaurant,Mexican Restaurant,Dumpling Restaurant,Bar,Coffee Shop,Park


<h2> Cluster Neighbourhoods

In [26]:
# set number of clusters
k = 5
torontogrpclustering = torontogrp.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(torontogrpclustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 0, 1], dtype=int32)

In [27]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

torontomerged = Torontodata

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
torontomerged = torontomerged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

torontomerged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Trail,Pub,Coffee Shop,Farmers Market,Falafel Restaurant,Event Space,Fast Food Restaurant,Dive Bar,Filipino Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Bookstore,Brewery,Bubble Tea Shop,Burger Joint
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Pizza Place,Fast Food Restaurant,Italian Restaurant,Pet Store,Gym,Pub,Movie Theater,Sandwich Place,Burrito Place,Burger Joint
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Chinese Restaurant,Latin American Restaurant,Fish Market,Seafood Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Bus Line,Dim Sum Restaurant,Swim School,Women's Store,Doner Restaurant,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant


Finally, let's visualize the resulting clusters

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(torontomerged['Latitude'], torontomerged['Longitude'], torontomerged['Neighborhood'], torontomerged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2> Examining Clusters
    

Cluster 1

In [ ]:
torontomerged.loc[torontomerged['Cluster Labels'] == 0, 
                  torontomerged.columns[[1] + list(range(5, torontomerged.shape[1]))]]